# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#import the cities _output.csv file from Part 1
cities_df = pd.read_csv('../output_data/cities_output.csv')
cities_df

,Unnamed: 0,country,city,lat,lon,date,cloudiness,humidity,max_temp,wind_speed
0,0,AU,busselton,-33.6500,115.3333,1643325133,0,75,60.04,10.42
1,1,IS,grindavik,63.8424,-22.4338,1643325134,100,93,39.67,23.02
2,2,RU,saryg-sep,51.5000,95.6000,1643325134,100,99,-8.27,0.85
3,3,BR,vila velha,-20.3297,-40.2925,1643325134,0,86,78.94,1.01
4,4,IN,veraval,20.9000,70.3667,1643325135,0,37,63.07,15.52
...,...,...,...,...,...,...,...,...,...,...
535,535,ML,gao,16.6362,1.6370,1643325972,7,20,62.80,13.04
536,536,ET,jijiga,9.3500,42.8000,1643325973,70,39,56.89,10.45
537,537,FI,kuusamo,65.9667,29.1833,1643325973,100,92,15.84,2.30
538,538,CA,hearst,49.6835,-83.6665,1643325973,100,90,-7.35,9.71


In [3]:
#drop the duplicate index column created during the csv process in Part !
cities_df.drop('Unnamed: 0', inplace=True, axis=1)


In [4]:
display(cities_df)

,country,city,lat,lon,date,cloudiness,humidity,max_temp,wind_speed
0,AU,busselton,-33.6500,115.3333,1643325133,0,75,60.04,10.42
1,IS,grindavik,63.8424,-22.4338,1643325134,100,93,39.67,23.02
2,RU,saryg-sep,51.5000,95.6000,1643325134,100,99,-8.27,0.85
3,BR,vila velha,-20.3297,-40.2925,1643325134,0,86,78.94,1.01
4,IN,veraval,20.9000,70.3667,1643325135,0,37,63.07,15.52
...,...,...,...,...,...,...,...,...,...
535,ML,gao,16.6362,1.6370,1643325972,7,20,62.80,13.04
536,ET,jijiga,9.3500,42.8000,1643325973,70,39,56.89,10.45
537,FI,kuusamo,65.9667,29.1833,1643325973,100,92,15.84,2.30
538,CA,hearst,49.6835,-83.6665,1643325973,100,90,-7.35,9.71


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [14]:
humidity = cities_df["humidity"].astype(float)
maxhumidity = cities_df["humidity"].astype(float).max()
locs_df = cities_df[["lat", "lon"]]
locs_df

,lat,lon
0,-33.6500,115.3333
1,63.8424,-22.4338
2,51.5000,95.6000
3,-20.3297,-40.2925
4,20.9000,70.3667
...,...,...
535,16.6362,1.6370
536,9.3500,42.8000
537,65.9667,29.1833
538,49.6835,-83.6665


In [6]:
maxhumidity

100.0

In [106]:
#https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html
#Best website for explaining how to use google maps
#Chose the center based on a value in dataframe--lat and lon
gmaps.configure(api_key = g_key)
figure = gmaps.figure(zoom_level=3,center=(48.8566,2.3522))
heat_layer = gmaps.heatmap_layer(locs_df, weights=humidity,dissipating=False,max_intensity=maxhumidity,point_radius=2)
figure.add_layer(heat_layer)
figure



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [107]:
cities_df

,country,city,lat,lon,date,cloudiness,humidity,max_temp,wind_speed
0,AU,busselton,-33.6500,115.3333,1643325133,0,75,60.04,10.42
1,IS,grindavik,63.8424,-22.4338,1643325134,100,93,39.67,23.02
2,RU,saryg-sep,51.5000,95.6000,1643325134,100,99,-8.27,0.85
3,BR,vila velha,-20.3297,-40.2925,1643325134,0,86,78.94,1.01
4,IN,veraval,20.9000,70.3667,1643325135,0,37,63.07,15.52
...,...,...,...,...,...,...,...,...,...
535,ML,gao,16.6362,1.6370,1643325972,7,20,62.80,13.04
536,ET,jijiga,9.3500,42.8000,1643325973,70,39,56.89,10.45
537,FI,kuusamo,65.9667,29.1833,1643325973,100,92,15.84,2.30
538,CA,hearst,49.6835,-83.6665,1643325973,100,90,-7.35,9.71


In [108]:
perfect_df = cities_df.loc[
    (cities_df['max_temp'] > 50) &
    (cities_df['max_temp'] < 75) &
    (cities_df['wind_speed']<=20) &
    (cities_df['humidity']<=50)].reset_index(drop=True).dropna()

display(perfect_df)



,country,city,lat,lon,date,cloudiness,humidity,max_temp,wind_speed
0,IN,veraval,20.9000,70.3667,1643325135,0,37,63.07,15.52
1,SA,najran,17.4924,44.1277,1643325135,11,27,60.26,1.61
2,NZ,te anau,-45.4167,167.7167,1643325137,71,42,66.15,0.92
3,AR,villa maria,-32.4075,-63.2402,1643325140,100,37,72.86,6.02
4,BF,tougan,13.0667,-3.0667,1643325146,3,15,72.34,9.86
5,NZ,mataura,-46.1927,168.8643,1643325207,52,35,64.47,1.34
6,ML,araouane,18.9048,-3.5265,1643325208,77,15,67.42,11.32
7,NE,arlit,18.7369,7.3853,1643325195,43,24,60.24,7.20
8,NZ,tuatapere,-46.1333,167.6833,1643325218,78,42,64.81,2.75
9,AR,viedma,-40.8135,-62.9967,1643325285,0,41,74.05,18.34


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [109]:
hotel_df = perfect_df

hotel_df['Hotel Name'] = ''

display(hotel_df.head())

,country,city,lat,lon,date,cloudiness,humidity,max_temp,wind_speed,Hotel Name
0,IN,veraval,20.9000,70.3667,1643325135,0,37,63.07,15.52,
1,SA,najran,17.4924,44.1277,1643325135,11,27,60.26,1.61,
2,NZ,te anau,-45.4167,167.7167,1643325137,71,42,66.15,0.92,
3,AR,villa maria,-32.4075,-63.2402,1643325140,100,37,72.86,6.02,
4,BF,tougan,13.0667,-3.0667,1643325146,3,15,72.34,9.86,


In [110]:
radius = 5000
place = 'hotel'

for index, row in hotel_df.iterrows():
    lat = row['lat']
    lon = row['lon']
    
    query_url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={lat}%2C{lon}&radius={radius}&type={place}&key={g_key}"

    result = requests.get(query_url).json()
    
    try:
        hotel_name = result['results'][1]['name']
    except:
        hotel_name = 'NA'
    
    hotel_df.at[index, 'Hotel Name'] = hotel_name


In [111]:
display(hotel_df)

,country,city,lat,lon,date,cloudiness,humidity,max_temp,wind_speed,Hotel Name
0,IN,veraval,20.9000,70.3667,1643325135,0,37,63.07,15.52,Lords Inn Somnath
1,SA,najran,17.4924,44.1277,1643325135,11,27,60.26,1.61,مركز صحي البلد
2,NZ,te anau,-45.4167,167.7167,1643325137,71,42,66.15,0.92,Kingsgate Hotel Te Anau
3,AR,villa maria,-32.4075,-63.2402,1643325140,100,37,72.86,6.02,Hotel Milenium
4,BF,tougan,13.0667,-3.0667,1643325146,3,15,72.34,9.86,Hôtel SMAK
5,NZ,mataura,-46.1927,168.8643,1643325207,52,35,64.47,1.34,Ellie's Villa
6,ML,araouane,18.9048,-3.5265,1643325208,77,15,67.42,11.32,NA
7,NE,arlit,18.7369,7.3853,1643325195,43,24,60.24,7.20,Station Total
8,NZ,tuatapere,-46.1333,167.6833,1643325218,78,42,64.81,2.75,Tuatapere Cemetery
9,AR,viedma,-40.8135,-62.9967,1643325285,0,41,74.05,18.34,Hotel Cristal


In [115]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lon"]]

In [116]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
figure.add_layer(marker_layer)

# Display figure
figure

Figure(layout=FigureLayout(height='420px'))